In [1]:
import gradio as gr
import cv2

from cnn2snn import set_akida_version, AkidaVersion
from akida import Model
import akida

import numpy as np

import time

import plotly.graph_objects as go

deviceCheckbox = ["CPU"]

/home/kmanninen/miniconda3/envs/akida_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-23 11:02:33.127569: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 11:02:33.181434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-23 11:02:33.181502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-2

In [2]:
def create_gauge(value):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        gauge={'axis': {'range': [0, 30]}},
        domain={'x': [0, 1], 'y': [0, 1]},
    ))
    fig.update_layout(width=400, height=300)
    return fig

In [3]:
# Softmax for an array of values
def softmaxArray(values):
    # Assuming array shape is (1, 1, 1, x), flatten to get the values
    values = values.ravel()
    exp_values = np.exp(values)
    sum_exp = np.sum(exp_values)
    softmax_values = exp_values / sum_exp
    return softmax_values

In [4]:
image_x = 96
image_y = 96
image_z = 3
labels = ["no person", "person"]
def decodeOutput(inp):
        global akida_model
        inp = cv2.resize(inp, (image_x, image_y))
        inp = inp.reshape((-1, image_x, image_y, image_z))
        timer_start = time.time()
        predictions = softmaxArray(akida_model.predict(inp))
        frame_time = time.time() - timer_start
        fps = 1 / frame_time if frame_time > 0 else 0
        confidences = {labels[i]: predictions[i] for i in range(len(predictions))}

        return confidences, fps

In [5]:
def classify_image(inp):

  confidences, fps = decodeOutput(inp)

  return confidences, create_gauge(round(fps, 2))

In [6]:
with set_akida_version(AkidaVersion.v2):
            global akida_model
            akida_model = Model("/home/kmanninen/akida2_fpgacloud_examples/models/akidanet_vww_i8_w8_a8.fbz")
            devices = akida.devices()
            if len(devices) > 0:
                print(f'Available devices: {[dev.desc for dev in devices]}')
                device = devices[0]
                print(device.version)
                try:
                    akida_model.map(device)
                    print(f"Mapping to Akida device {device.desc}.")
                    mappedDevice = "FPGA"
                except Exception as e:
                    print("Model not compatible with FPGA. Running on CPU.")
                    mappedDevice = "CPU"
            else:
                print("No Akida devices found, running on CPU.")
                mappedDevice = "CPU"

No Akida devices found, running on CPU.


In [7]:
deviceCheckbox = [mappedDevice]

In [8]:
theme = gr.themes.Base(
    text_size="sm",
    spacing_size="sm",
    radius_size="sm",
)

with gr.Blocks(
    title="Brainchip",
    fill_width=True,
    fill_height=True,
    delete_cache=[180, 600],
    theme=theme
) as demo:
    with gr.Tabs():
        with gr.Tab("Image Classification") as image_classification_tab:
            gr.Markdown("""
                <h1 style="text-align: center;">Akida Cloud</h1>
                <br>
                """)
            with gr.Row():
                gr.Markdown("## Image Classification")
            with gr.Row():
                with gr.Column():
                    input_img = gr.Image(sources=["webcam"], type="numpy")
                with gr.Column():
                    checkbox_group = gr.CheckboxGroup(choices=["FPGA", "CPU"], value=deviceCheckbox, label="Device", interactive=False)
                    output_label = gr.Label(num_top_classes=3)
                    plot = gr.Plot(label="Frames per second")
                dep = input_img.stream(classify_image, [input_img], [output_label, plot],
                                        time_limit=30, stream_every=0.1, concurrency_limit=30)        


if __name__ == "__main__":
    demo.launch(allowed_paths=["img/"])

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
